# Исследование надежности заемщиков.

<b>Описание проекта</b>\
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Обзор данных 
<b>Описание данных</b>\
•	children — количество детей в семье\
•	days_employed — общий трудовой стаж в днях\
•	dob_years — возраст клиента в годах\
•	education — уровень образования клиента\
•	education_id — идентификатор уровня образования\
•	family_status — семейное положение\
•	family_status_id — идентификатор семейного положения\
•	gender — пол клиента\
•	income_type — тип занятости\
•	debt — имел ли задолженность по возврату кредитов\
•	total_income — ежемесячный доход\
•	purpose — цель получения кредита

Для начала необходимо импортировать библиотеку pandas и открыть датафрейм для дальшейней работы.

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Чтобы понять общую картину с какими данными предстоит работу, вызываем .info()

In [2]:
df.info() #получение общей информации о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данная операция сразу даёт нам понять что в данных присутствую пропуски, так же видно из выведеной части таблицы наличие аномалий.

### Заполнение пропусков

Сперва определим в каких столбцах присутствуют пропуски

In [3]:
df.isnull().sum() #в столбцах days_employed и total_income присутствуют пустые значения

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения: days_employed и total_income. Чтобы понять как бороться с пропусками, стоит определить их долю в предоставленных данных

In [4]:
null_percent = df['days_employed'].isnull().sum() / df.shape[0]  #доля пропущеных значений
print(f'Доля пропущенных значений составляет: {null_percent:.1%}')

Доля пропущенных значений составляет: 10.1%


Доля составляет значимую часть предоставленных данных, в связи с этим удаление может привести к неправильным результатам исследования. Для этого необходимо заменить пропущенные данные на медианные значения по столбцу. Замена на медиану в данном датасете будет предпочтительнее среднего арифметического, т.к. вид деятельности, например, сотрудник и госслужащий поделены абстрактно, и не показывают должность или сферу деятельности.\
Для того, чтобы анализ получился точным, медиану нужно брать по виду деятельности, т.к. отработанные дни и заработная плата у пенсионера и работающего человека будет отличаться.\
В данном случае я решил использовать циклы для замены данных по виду деятельности на медианы, которые им соответствуют.

Сначала необходимо составить список уникальных значений видов деятельности и новый список для расчета медиан по ним.

In [5]:
list_income_type = df['income_type'].unique() #список видов деятельности

In [6]:
median_days_employed = [] #список медиан по видам деятельности
for lists in list_income_type:
    median_days_employed.append(df[df['income_type'] == lists]['days_employed'].median())

In [7]:
median_total_income = [] #список медиан по доходу
for lists in list_income_type:
    median_total_income.append(df[df['income_type'] == lists]['total_income'].median())

Далее используем цикл для замены данных. Первый работает цикл для устранения пропусков по столбцу с днями, второй - зарплате.

In [8]:
for i in range(len(list_income_type)): #замена пропущеных дней на медианы
    df.loc[(df['days_employed'].isna() == True)&(df['income_type'] == list_income_type[i]), 'days_employed'] = median_days_employed[i]

In [9]:
for i in range(len(list_income_type)): #замена пропущеных значений на медианый доход
    df.loc[(df['total_income'].isna() == True)&(df['income_type'] == list_income_type[i]), 'total_income'] = median_total_income[i]

Используем .isna() чтобы убедиться в успешной замене пропусков. 

In [10]:
df.isna().sum() #проверка на успех

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Проверка данных на аномалии и исправления.

На первый взгляд в данных встречаются аномалии. Одна из них - отрицательное количество дней трудового стажа в столбце days_employed. Для исправления сначала отбираем только те значения, которые меньше нуля.

In [11]:
days_anomaly = df[df['days_employed'] < 0].count() #проверка на количество отрицательных значений
days_anomaly

children            17667
days_employed       17667
dob_years           17667
education           17667
education_id        17667
family_status       17667
family_status_id    17667
gender              17667
income_type         17667
debt                17667
total_income        17667
purpose             17667
dtype: int64

Отрицательных значений в таблице много, из этого можно сделать вывод что были ошибки при сборе данных или других манипуляций с данными. В данной ситуации будет разумно взять все отрицательные значения в модуль, чтобы исправить аномалию.

In [12]:
df['days_employed'] = df['days_employed'].apply(abs) #взятие в модуль всех значений

In [13]:
days_anomaly = df[df['days_employed'] < 0].count() #проверка на успех
days_anomaly

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

От отрицательных значений удалось избавиться.

В days_employed есть слишком большие значения. Сначал определим сколько строк содержат такие значения и к каким категорям занятости относятся. Для того чтобы определить какие строки считать слишком большими, попробуем узнать в каких строках кол-во рабочих дней превышает возраст человека

In [14]:
df.loc[df['days_employed'] >= (df['dob_years'] * 360)].value_counts('income_type')

income_type
пенсионер      3856
сотрудник        55
компаньон        20
госслужащий       6
безработный       2
dtype: int64

Из таблицы видно, что ошибочка информация превалирует у пенсионеров. В данном случае можно использовать несколько методов: заменить на среднее по ячейке, обьявить что данные отсутствуют или просто проигнорировать. Так как в исследовании эти данные нам не пригодятся, чтобы не ранить нужные нам данные в других столбцах считаю верным не трогать эти данные.

Есть еще несколько аномалия столбце 'children'. Сперва необходимо узнать количество уникальных значений.

In [15]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Как видно из примера выше в данных присутствуют отрицательные значения и с 20 детьми. Можно предположить, что в данных столбцах просто отсутствуют значения, поэтому сперва нужно заменить значения в столбце на пропуски и их удалить строки.

В этом случае отрицательных значений немного. Можно предположить, что в данных столбцах просто отсутствуют значения, поэтому сперва нужно заменить столбцы на пропуски и их удалить.

In [16]:
df.loc[(df['children'] < 0)| (df['children'] == 20)] = None #изменение отрицательных значений на пропуск

In [17]:
df = df.dropna(subset=['children'])  #удаление строк с пропущеными данными

In [18]:
df['children'].isna().sum() #проверка на успех

0

После проверки видно что столбцы удалились.

### Изменение типов данных.

Для удобства чтения таблицы необходимо заменить типы  данных таблицы на целочисленный. В нескольких столбцах после операции присвоения пустых значений типы данных поменялись с int на float. Сделаем цикл для исправления изменений:

In [19]:
float_column = ('total_income', 'children', 'dob_years', 'education_id', 'family_status_id', 'debt') #цикл по замене типа данных
for column in float_column:
    df[column] = df[column].astype('int')

In [20]:
df.info() #проверка на успех

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  int64  
 11  purpose           21402 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


После данной операции все строки стали выглядять приятнее для исследования.

### Удаление дубликатов.

Далее следуем осмотреть таблицу на явные и неявные дубликаты. Для этого используем .duplicated() для явных дубликатов и .unique() для неявных.

In [21]:
df.duplicated().sum() #проверка на наличие дубликатов

54

In [22]:
df = df.drop_duplicates().reset_index(drop=True) #удаление дубликатов с изменением индекса

In [23]:
df.duplicated().sum() #проверка на успех

0

После проверки видно, что явныз дубликатов не осталось. По голове таблицы сразу было заметно, что в столбце 'education' находятся значения с разным регистром. Нужно проверить столбец на уникальные значения

In [24]:
df['education'].unique() #проверка на неявные дубли

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Здесь чтобы убрать неявные дубликаты следует изменить регистр всего столбца на нижний и тогда дубликаты пропадут.

In [25]:
df['education'] = df['education'].str.lower() #устранение неявных дублей

In [26]:
df['education'].unique() #проверка на успех

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Последней операцией проверили, что все значения убрались и дублей нет. Эти проблемы вероятно могли возникнуть при сборе информации от разных источников, в связи с чем каждый описывал значение по своему.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Данный датафрейм содержить значения, которые легко можно преобразовавть в отдельные словари, чтобы информация читалась удобнее. Столбцы 'education', 'education_id' и 'family_status', 'family_status_id' можно вывести отдельно, т.к. содержат похожую информацию. После выноса данных столбцов в отдельные таблицы education и family, можно удалить дублирующие столбцы из основной таблицы, а для необходимости обращаться к новым справочникам.

In [27]:
education = df[['education', 'education_id']] #датафрейм образование
education = education.drop_duplicates().reset_index(drop=True)

In [28]:
family = df[['family_status', 'family_status_id']] #датафрейм семья
family = family.drop_duplicates().reset_index(drop=True)

Далее удаляем стобцы:

In [29]:
df.drop(['family_status', 'education'], axis=1, inplace=True)
df.head() #удаление столбцов

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


На выходе получилась удобная таблица без загромождения.

### Категоризация дохода.

Чтобы удобно категоризовать доход, можно добавить дополнительны столбец total_income_category и присвоить каждой строке свою оценку, опираясь на доход:
0–30000 — 'E';\
30001–50000 — 'D';\
50001–200000 — 'C';\
200001–1000000 — 'B';\
1000001 и выше — 'A'.\
Для этого нужно создать функцию, которая будет присваивать значение и применить её к каждой строке датафрейма с помощью .apply()

In [30]:
def categorisation(income): #категоризация дохода
    if 0 <= income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'

In [31]:
df['total_income_category'] = df['total_income'].apply(categorisation) #проверка результата
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [32]:
df['total_income_category'].isna().sum() #

0

Новая функция справилась с задачей. Чтобы проверить заполнился ли весь столбец, была проведена проверка через .isna()

### Категоризация целей кредита.

Для категоризации целей кредита, нужно создать функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, и задаст новые категории:\
Здесь нужно использовать собственную функцию и метод .apply()

In [33]:
df['purpose'].unique() #поиск уникальныз значений

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Из уникальных значений видно, что присутствует 4 основные категории:\
'операции с автомобилем',\
'операции с недвижимостью',\
'проведение свадьбы',\
'получение образования'.\
Для каждой из них можно найти общее слово или корень и сделать логический фильтр при присваивании значения.

In [34]:
def category(purpose): #формула для присвоения категории
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
    return purpose

In [35]:
df['purpose_category'] = df['purpose'].apply(category) #проверка результата
df['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

In [36]:
df['purpose_category'].isna().sum()

0

Категории были присвоены корректно, пропусков нет.

### Ответы на вопросы.

Для ответов на вопросы выводы сделаны на основе сводных таблик из датасета.

##### Вопрос 1: •	Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Вывод 1: 
Для людей категории с 5 детьми видим, что в данной таблице есть всего 9 таких строк, поэтому выводы по ней будут не репрезентативны. В остальных случаях можно сделать вывод, что у клиентов без детей вероятность задолженности ниже, чем у клиентов с детьми, а количество детей почти не влияет на вероятность задолженности.

In [37]:
children_pivot = pd.pivot_table(df, index='children', values='debt', aggfunc=['mean', 'count']) #сводная таблица по детям и возврату кредита
children_pivot.columns = ['mean', 'count']
children_pivot

,mean,count
children,,
0,0.075353,14107
1,0.092327,4809
2,0.094542,2052
3,0.081818,330
4,0.097561,41
5,0.000000,9


##### Вопрос 2: • 	Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Вывод 2: 
В таблице видна зависимость возврата кредита в срок и семейного положения. Лучше всего дела обстаят у категории вдовец/вдова, в которой только 6,6% кредитов не возвращены в срок. У тех кто в разводе и женат/замужем ситуация немного хуже: 7,1% и 7,6% соответственно. Ненадежными будут: те кто не замужем/не женат и кто в гражданском браке, у них невозврат составляет 9,8% и 9,4%.\
Можно сделать вывод, что у клиентов с семеный положением не женат/не замужем и в гражданском браке вероятность задолженности выше, чем у остальных категорий. Из этого следует, что здесь присутствует зависимость между семейным положением и возвратом кредита в срок.

In [38]:
family_status_pivot = pd.pivot_table(df, index='family_status_id', values='debt', aggfunc=['mean', 'count']) #сводная таблица по семейному положению и возврату кредита
family_status_pivot.columns = ['mean', 'count']
family_status_pivot

,mean,count
family_status_id,,
0,0.075575,12266
1,0.092861,4146
2,0.066246,951
3,0.070648,1189
4,0.097639,2796


In [39]:
family #словарь для подсказки

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


##### Вопрос 3: •	Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Вывод 3:,
По сводной таблице можем заметить, что доля невозврата колеблется между разными категорями дохода. Для этого сравним две большие группы D и C. Например в группе D (клиенты с доходом от 30000 до 50000) процент невозврата 6,0%, тогда как в группе С (от 50000 до 200000) процент невозврата составляет 8,4%.Если рассмотреть группы с более высоким доходом A и B, то они показывают  разный результат: 8,0% и 7,0% соотв.
Еще можно рассмотреть группы D и B, где при более высоком доходе процент невозврата выше (6.0% и 7.0%
Делаем вывод, что уровень дохода не влияет на вероятность возврата долга.

In [40]:
income_pivot = pd.pivot_table(df,               #средний доход тех кто вернул кредит и кто не вернул в срок
               index='total_income_category', 
               values='debt', 
               aggfunc=['mean', 'count']) 
income_pivot.columns = ['mean', 'count']
income_pivot

,mean,count
total_income_category,,
A,0.080000,25
B,0.070602,5014
C,0.084891,15938
D,0.060172,349
E,0.090909,22


##### Вопрос 4: •	Как разные цели кредита влияют на его возврат в срок?

##### Вывод 4:
Из таблицы видно, что операции с автомобилем и получение образования являются ненадежными в вопросе возврата средств в срок (9,4% и 9,2% соотв.), а цели операции с недвижимостью и проведение свадьбы надежнее (7,2% и 8,0% соотв.)
По категориям видно, что шанс возврата по разным целям отличается, из чего сделаем вывод, что они влияют на возврат кредита.

In [41]:
pd.pivot_table(df, index='purpose_category', values='debt')

,debt
purpose_category,
операции с автомобилем,0.093436
операции с недвижимостью,0.072531
получение образования,0.092504
проведение свадьбы,0.078744


##### Общий вывод: 
Исследуя предоставленные данные, можно сделать вывод:
Семейное положение и цель кредита могут оказать влияние на вероятность задолженности по кредиту, а вот уровень заработной платы на этот показатель влияния не оказывает. У клиентов без детей вероятность задолженности ниже, чем у клиентов с детьми, количество детей почти не влияет на вероятность задолженности.
